In [1]:
import pandas as pd
import numpy as np
import pymorphy2
from matplotlib.colors import ListedColormap
from numpy.random import choice
from matplotlib import pyplot as plt

import re

from gensim.models import FastText,KeyedVectors,fasttext

from sklearn import metrics

from scipy import spatial

import torch

from transformers import AutoTokenizer, AutoModel

'''import tensorflow as tf
from tensorflow.keras.layers import Reshape,BatchNormalization,MaxPooling2D,Lambda
from tensorflow.keras.layers import Dense,Activation,Reshape,Conv2D,LeakyReLU,concatenate,Flatten
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop, Adam
'''
from sklearn.model_selection import train_test_split
from sklearn import cluster

import seaborn as sns
import umap

C:\Anaconda\envs\torch\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Search engine

FastText documents embeddings vs questions

document=questions+answers on one theme

In [ ]:
questions=[]
documents=[]
class_num=len(faq1)
questions2class=dict()
questions_classes=[]



for i,document in enumerate(faq1):
    
    temp_questions=[]
    for line in document:
        question=line[0]
        temp_questions.append(question)
        questions2class[question]=i
    questions.append(temp_questions)
        
        
    temp_documents=[]
    for line in document:
        temp_documents.append(line[0])
        temp_documents.append(line[1])

        
    documents.append(temp_documents)




In [ ]:
documents_embs=np.zeros((len(documents),300))
for i,document in enumerate(documents):
    documents_embs[i]=sentence_embedding(ft_model,document)
    
questions_embs=[]
for i,question_document in enumerate(questions):
    temp_questions_embs=[]
    for question in question_document:
        temp_questions_embs.append(sentence_embedding(ft_model,[question]))
    questions_embs.append(temp_questions_embs)
    

In [ ]:
for i,document in enumerate(documents_embs):

    pairs_number=len(questions_embs[i])
    pairs=[]
    labels=[]
    #  true pairs
    for question in questions_embs[i]:
        pairs.append([document,question])
        labels.append(1)
        
    # false document
    for j in range(pairs_number):
        flag=True
        rnd_indx=np.random.randint(0,len(documents_embs))
        while  rnd_indx==i:
                rnd_indx=np.random.randint(0,len(documents_embs))
        pairs.append([documents_embs[rnd_indx],question])
        labels.append(0)
        
    # false questions
    for j in range(pairs_number):
        flag=True
        rnd_indx=np.random.randint(0,len(documents_embs))
        while rnd_indx==i:
                rnd_indx=np.random.randint(0,len(documents_embs))
                
        rnd_indx_question=np.random.randint(0,len(questions_embs[rnd_indx]))
        pairs.append([document,questions_embs[rnd_indx][rnd_indx_question]])
        labels.append(0)
    

In [ ]:
pairs=np.array(pairs).astype('float32')
labels=np.array(labels).astype('float32')
x_train,x_test,y_train,y_test=train_test_split(pairs,labels)


In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
    
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return K.mean( (1 - y_true) * square_pred+y_true*margin_square)
    
    return contrastive_loss

In [ ]:
def get_model():
    #
    #   it is also good idea to use VGG model, but it requires powerfull GPU
    #   model = Sequential(VGG16(weights='imagenet', include_top=False, input_shape=image_shape).layers)
    #
    model=Sequential()
    
    model.add(Reshape((15,20,1),input_shape=emb_shape))
#    model.add(Conv2D(32,(3,3)))
#    model.add(BatchNormalization())
        
    
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(0.2))

    return model

In [ ]:
emb_shape=(300,)
base_network = get_model()

input_a = Input(shape=emb_shape)
vect_output_a = base_network(input_a)

input_b = Input(shape=emb_shape)
vect_output_b = base_network(input_b)

x = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([vect_output_a, vect_output_b])
output= Dense(1,activation='sigmoid')(x)

# specify the inputs and output of the model
model = Model([input_a, input_b], output)

In [ ]:
optim = RMSprop(  learning_rate=0.015)
#optim = Adam(  learning_rate=0.015)
model.compile(loss=contrastive_loss_with_margin(margin=1), optimizer=optim)
history = model.fit([x_train[:,0],x_train[:,1]], 
                    y_train, 
                    epochs=30, 
                    batch_size=1,
                    validation_split=0.2)

In [ ]:
model.summary()

In [ ]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() > 0.5
    return round(np.mean(pred == y_true),4)

In [ ]:
loss = round(model.evaluate(x=[x_test[:,0],x_test[:,1]], y=y_test),4)

y_pred_train = model.predict([x_train[:,0],x_train[:,1]])
train_accuracy = compute_accuracy(y_train, y_pred_train)

y_pred_test = model.predict([x_test[:,0], x_test[:,1]])
test_accuracy = compute_accuracy(y_test, y_pred_test)

print("Loss = {}, Train Accuracy = {} Test Accuracy = {}".format(loss, train_accuracy, test_accuracy))

## Paragraph ranker

In [ ]:
from deeppavlov import build_model, configs

## Question Answering

[Reference](https://medium.com/swlh/fine-tuning-bert-for-text-classification-and-question-answering-using-tensorflow-framework-4d09daeb3330#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjFiZjhhODRkM2VjZDc3ZTlmMmFkNWYwNmZmZDI2MDcwMWRkMDZkOTAiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2MjU5MDYxNDIsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwNjU4MDUxNjE5NzU3NTk0MTk2NCIsImhkIjoibWllbS5oc2UucnUiLCJlbWFpbCI6ImRna2FncmFtYW55YW5AbWllbS5oc2UucnUiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXpwIjoiMjE2Mjk2MDM1ODM0LWsxazZxZTA2MHMydHAyYTJqYW00bGpkY21zMDBzdHRnLmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwibmFtZSI6ItCU0LDQstC40LQg0JrQsNCz0YDQsNC80LDQvdGP0L0iLCJwaWN0dXJlIjoiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EtL0FPaDE0R2hGdEZGSkZMaDlyUFV0QXdZLXlyZWlRczJObjEyQ2xpc3hmX3hQPXM5Ni1jIiwiZ2l2ZW5fbmFtZSI6ItCU0LDQstC40LQiLCJmYW1pbHlfbmFtZSI6ItCa0LDQs9GA0LDQvNCw0L3Rj9C9IiwiaWF0IjoxNjI1OTA2NDQyLCJleHAiOjE2MjU5MTAwNDIsImp0aSI6IjhiYmVjYmRjYWRiZTAyZDZkNjQ1MTMxY2Q2NzJhZTI4MWViMTdkNWYifQ.LRr773uwpuGgpYnO7WsltRMVbC3JdFg7DBIlPtnLhN11OpKyNZ5X3y2ZGLx72_tJaSXZS52xdLcEsLuM4Tk1Ta_4ifnuAMkkeJMYJct0DpJOqXGYWS9S2atl8JLvbQaLyNrseNHneLtzRgtj--Htk1Lq0red-VmGxO849tzhKfjfQDPw-PsKxnhKMxHoEHJs91z-djL_L8ATQ6p86TcSWzWGsi4Ya69TmkMiRw-W2eMaZjN11gjSXUCnJKUUC122fluOmDzsGnUbtpoSGD-86mTa368FTFLbQSC7M8SGiZKo_pIZCcKrGjBQM-bJFAMs3-I8J7nM6782Vlh_CZiBew)

[просто ссылка с хорошей инфой]( https://lilianweng.github.io/lil-log/2020/10/29/open-domain-question-answering.html )